In [1]:
import json
from pathlib import Path

In [2]:
project_root = Path.cwd().parent
project_root

WindowsPath('d:/workspace/docent')

In [3]:

# Read the relic_index.json file
with open(project_root / "data" / "database" / "relic_index.json", "r", encoding="utf-8") as f:
    relic_index = json.load(f)

In [4]:
import re

HANJA_RE = re.compile(
    r"["
    r"\u4E00-\u9FFF"  # 기본
    r"\u3400-\u4DBF"  # 확장 A
    r"\uF900-\uFAFF"  # 호환 한자
    r"\U00020000-\U0002A6DF"  # 확장 B
    r"\U0002A700-\U0002B73F"  # 확장 C–D
    r"\U0002B740-\U0002B81F"  # 확장 E
    r"]+"
)

def clean_text(text: str, replace_with: str = "") -> str:
    text = re.sub(r"\([^)]*\)|,", "", text)
    text = HANJA_RE.sub(replace_with, text)
    return text


In [5]:
# Get project root directory
import sys
project_root = Path.cwd().parent
sys.path.append(project_root)

In [6]:
sys.path

['C:\\Users\\bigbl\\AppData\\Local\\Programs\\Python\\Python312\\python312.zip',
 'C:\\Users\\bigbl\\AppData\\Local\\Programs\\Python\\Python312\\DLLs',
 'C:\\Users\\bigbl\\AppData\\Local\\Programs\\Python\\Python312\\Lib',
 'C:\\Users\\bigbl\\AppData\\Local\\Programs\\Python\\Python312',
 'd:\\workspace\\docent\\venv_dcnt',
 '',
 'd:\\workspace\\docent\\venv_dcnt\\Lib\\site-packages',
 'd:\\workspace\\docent\\venv_dcnt\\Lib\\site-packages\\win32',
 'd:\\workspace\\docent\\venv_dcnt\\Lib\\site-packages\\win32\\lib',
 'd:\\workspace\\docent\\venv_dcnt\\Lib\\site-packages\\Pythonwin',
 WindowsPath('d:/workspace/docent')]

In [7]:
from dataclasses import dataclass, field
import numpy as np
import os
from pathlib import Path
from openai import OpenAI
import dill
import json
import re

upstage = OpenAI(
    api_key=os.getenv("UPSTAGE_API_KEY"), base_url="https://api.upstage.ai/v1"
)


@dataclass(slots=True)
class DocEmbedding:
    id: str
    doc: str
    embedding: list[float] | None = None


@dataclass(slots=True)
class Similarity:
    id: str
    doc: str
    score: float = 0
    collection_name: str = "" 


class Collecton:

    _instances = {}

    def __new__(cls, name: str):
        if name not in cls._instances:
            instance = super().__new__(cls)
            cls._instances[name] = instance
        return cls._instances[name]

    def __init__(self, name: str):
        if getattr(self, "_initialized", False):
            return
        
        self.name = name
        self.file_path = project_root / "data" / "vector_store" / f"{name}"
        self.index: dict[str, DocEmbedding] = {}
        self._initialized = True

    def load(self) -> "Collecton":
        with open(f"{self.file_path}_meta.json", "r", encoding="utf-8") as f:
            docs_list = json.load(f)

        embeddings_array = np.load(f"{self.file_path}_embeddings.npy")

        for doc, embedding in zip(docs_list, embeddings_array):
            self.index[doc["id"]] = DocEmbedding(
                id=doc["id"],
                doc=doc["doc"],
                embedding=embedding.tolist()
            )
            
        return self        

    def add_doc(self, id: str, doc: str):
        self.index[id] = DocEmbedding(id=id, doc=doc)

    def build(self):
        doc_embeddings_all = list(self.index.values())
        doc_embeddings_chunks = [doc_embeddings_all[i:i+100] for i in range(0, len(doc_embeddings_all), 100)]

        doc_all_list = []
        embedding_all_list = []
        for doc_embeddings in doc_embeddings_chunks:
            docs = [doc_embedding.doc for doc_embedding in doc_embeddings]
            embeddings = self._get_embeddings(docs)
            for doc_embedding, embedding in zip(doc_embeddings, embeddings):
                doc_embedding.embedding = embedding
                doc_all_list.append({"id": doc_embedding.id, "doc": doc_embedding.doc})
                embedding_all_list.append(embedding)

        embedding_np_array = np.array(embedding_all_list)
        np.save(f"{self.file_path}_embeddings.npy", embedding_np_array)        
    
        with open(f"{self.file_path}_meta.json", "w", encoding="utf-8") as f:
            json.dump(doc_all_list, f, ensure_ascii=False, indent=2)

    def query(self, query: str, cutoff=0.4, top_k: int = 60) -> dict[int, Similarity]:
        query_embedding = self._get_embeddings(query)[0]
        similarities: list[Similarity] = []
        for doc_embedding in self.index.values():
            score = np.dot(query_embedding, doc_embedding.embedding) / (
                np.linalg.norm(query_embedding)
                * np.linalg.norm(doc_embedding.embedding)
            )
            if score < cutoff:
                continue
            similarities.append(
                Similarity(
                    id=doc_embedding.id,
                    doc=doc_embedding.doc,
                    score=float(score),
                    collection_name=self.name
                )
            )

        similarities = sorted(similarities, key=lambda x: x.score, reverse=True)[:top_k]
        return similarities

    def _get_embeddings(self, texts: list[str]) -> list[float]:
        embeddings = upstage.embeddings.create(input=texts, model="embedding-query")
        return [embedding_data.embedding for embedding_data in embeddings.data]            

    def __len__(self) -> int:
        return len(self.index)

In [71]:
Collecton._instances.pop("title", None)
title_collection = Collecton("title")
for i, (key, value) in enumerate(relic_index.items()):    
    #if i >= 2: break
    doc=f"{clean_text(value['label']['명칭']).strip()}, {clean_text(value['label']['다른명칭']).strip()}"
    title_collection.add_doc(id=key, doc=doc)

len(title_collection)

433

In [15]:
#title_collection.build()
title_collection = Collecton("title")
len(title_collection)

list(title_collection.index.keys())[-1]

title_collection.index['36560665']


DocEmbedding(id='36560665', doc='강세황 초상 자필본, ', embedding=[-0.00960540771484375, -0.0308685302734375, -0.0203094482421875, 0.003551483154296875, 0.0001423358917236328, -0.01009368896484375, 0.0195770263671875, -0.0156707763671875, -0.01396942138671875, 0.005611419677734375, -0.006496429443359375, 0.0086669921875, 0.0116119384765625, -0.0219268798828125, 0.0009202957153320312, -0.00042629241943359375, -0.0244598388671875, 0.00970458984375, 0.0016088485717773438, 0.00934600830078125, -0.00974273681640625, -0.0227508544921875, 0.004032135009765625, -0.004314422607421875, 0.009674072265625, 0.006481170654296875, -0.002452850341796875, -0.002132415771484375, -0.0007433891296386719, 0.0086517333984375, 0.0276336669921875, -0.02740478515625, 0.010986328125, 0.031646728515625, 0.00850677490234375, 0.01861572265625, -0.0239105224609375, 0.01369476318359375, 0.00716400146484375, 0.0187835693359375, -0.0201568603515625, 0.0121612548828125, -0.00946044921875, 0.00794219970703125, -0.01079559326171

In [10]:
title_collection.load()
print("last key", list(title_collection.index.keys())[-1])
resp =title_collection.query("감산사 석조미륵보살입상, 국보 경주 감산사 석조 미륵보살 입상")
print(resp[0].score)
print(resp[1].score)

last key 36560665
0.99999758197161
0.8690045986054047


In [103]:
Collecton._instances.pop("content", None)
content_collection = Collecton("content")
for i, (key, value) in enumerate(relic_index.items()):    
    #if i >= 2: break
    if not value['content']:
        doc=f"{clean_text(value['label']['명칭']).strip()}, {clean_text(value['label']['다른명칭']).strip()}"
    else:
        doc=f"{clean_text(value['content']).strip()}"
    
    content_collection.add_doc(id=key, doc=doc)

len(content_collection)

433

In [106]:
#content_collection.build()

In [107]:
content_collection.load()
print("last key", list(content_collection.index.keys())[-1])
resp = content_collection.query("신체와 광배는 하나의 돌로 제작하고 별도로 제작한 대좌에 결합시켰다. 이러한 형식은 감산사 절터에서 함께 수습된 <아미타불>과 같다. 머리에는 높은 보관을 썼는데 중앙에 화불이 있다. 얼굴은 갸름하나 살이 올라 있고 눈과 입에 미소가 어려 있다. 목에는 삼도가 뚜렷하며 목걸이 팔찌 영락 장식 등으로 신체를 화려하게 장식하고 있다. 오른손은 자연스럽게 내려뜨리고 있고 왼손은 들어 올려 손바닥을 보이고 있다. 팔목에는 천의가 걸쳐져 있는데 법의는 얇아서 신체의 풍만하고 유려한 곡선을 더욱 살려주고 있다. 광배는 배모양에 신체를 모두 감싸는 주형거신광으로 세 가닥의 선으로 두광과 신광을 구분하였다. 광배 뒷면에는 명문이 새겨져 있는데 이를 통해 719년 김지성이 돌아가신 어머니를 위해 조성한 미륵보살상임을 알 수 있다. 표현이 사실적이고 관능적인 모습을 한 통일신라 8세기 불상의 대표적인 사례이다.")
print(resp[0].score)
print(resp[1].score)

last key 36560665
0.9999971392296716
0.837087236523226


In [116]:
Collecton._instances.pop("description", None)
description_collection = Collecton("description")
for i, (key, value) in enumerate(relic_index.items()):    
    doc=f"{clean_text(value['image_description']).strip()}"    
    description_collection.add_doc(id=key, doc=doc)

len(description_collection)

433

In [112]:
#description_collection.build()

In [125]:
description_collection.load()
print("last key", list(description_collection.index.keys())[-1])
resp = description_collection.query("이 이미지는 전통적인 불교 조각상으로 타원형의 배경 안에 서 있는 보살상을 보여줍니다. 조각상은 석재로 만들어졌으며 옅은 황토색 또는 베이지 톤을 띠고 있습니다. 보살은 정면을 바라보며 서 있고 머리에는 왕관을 쓰고 있으며 세부적인 의복과 장신구를 착용하고 있습니다. 뒤로는 불꽃 모양의 배경이 있으며 조각상은 연꽃 모양의 받침대 위에 놓여 있습니다. 조각의 디테일은 섬세하고 우아하며 불교 예술의 전형적인 특징을 잘 보여줍니다.")
print(resp[0].score)
print(resp[1].score)

last key 36560665
0.9999978014469297
0.8121019154156572
